# Setup and Authentication
This section of the code performs necessary setup steps to prepare your environment for interacting with Google Cloud Platform (GCP) services, specifically Google BigQuery and Google Maps Platform.

In [ ]:
!pip install google-cloud-bigquery google-maps-places


This line uses the !pip install command, which allows you to run shell commands directly within your Jupyter Notebook. It installs two Python libraries:

google-cloud-bigquery: This library provides the tools to interact with Google BigQuery, a fully managed data warehouse.
google-maps-places: This library provides access to the Google Maps Platform Places API, which allows you to search for places and retrieve details.


In [3]:
!gcloud auth application-default login

This command uses the !gcloud utility to authenticate your application to access Google Cloud resources. gcloud auth application-default login is a standard way to set up credentials for applications running on your local machine or development environment to interact with GCP services. This command will typically open a web browser and prompt you to log in with your Google account and authorize access.

In [1]:
PROJECT_ID = "ml-demo-384110"
BIGQUERY_PROJECT_ID = PROJECT_ID
BIGQUERY_DATASET_ID = 'burger_king_reviews_dataset'
BIGQUERY_TABLE_ID = 'france_reviews_v2'
BIGQUERY_TABLE_PLACE_ID = 'places'
BIGQUERY_TABLE_PLACE_DETAILS = 'places_details_v2'
BIGQUERY_DATASET_LOCATION = 'EU' # Or your preferred GCP region, e.g., 'US'


These lines define several variables that store important information related to your Google Cloud Project and BigQuery setup.

PROJECT_ID: Your Google Cloud Project ID. This is a unique identifier for your project within GCP.
BIGQUERY_PROJECT_ID: This variable is set to the same value as PROJECT_ID, explicitly indicating the project to be used for BigQuery operations.
BIGQUERY_DATASET_ID: The ID of the BigQuery dataset you will be using. A dataset is a top-level container in BigQuery for organizing tables and views.
BIGQUERY_TABLE_ID: The ID of a specific BigQuery table within the dataset. This variable is used later, although other table IDs are also defined and used.
BIGQUERY_TABLE_PLACE_ID: Another variable for a BigQuery table ID, possibly intended for a table storing place IDs.
BIGQUERY_TABLE_PLACE_DETAILS: A variable for the ID of a BigQuery table that will store detailed information about places. This table is used later in the code.
BIGQUERY_DATASET_LOCATION: Specifies the geographic location where your BigQuery dataset will be stored. In this case, it's set to 'EU', but you can change it to your preferred region.

In [4]:

!gcloud config set billing/quota_project $PROJECT_ID
!gcloud auth application-default set-quota-project $PROJECT_ID

Updated property [billing/quota_project].

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "ml-demo-384110" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


## Set your Google map API Key

In [4]:
from google.colab import userdata
# Get Google map API KEY from your secret.

# TODO: Set your own.
API_KEY = userdata.get('API_KEY')

## Python librairy

In [5]:
from google.maps import places_v1
from google.type import latlng_pb2


client = places_v1.PlacesAsyncClient(
  # Instantiates the Places client, passing the API key
  client_options={"api_key": API_KEY}
)

async def text_search(search_query):
  client = places_v1.PlacesAsyncClient()
  # Build the request
  request = places_v1.SearchTextRequest(
      text_query=search_query,
      language_code="fr",
  )
  # Set the field mask
  fieldMask = "places.formattedAddress,places.displayName,places.id,places.location"
  #fieldMask = "places.formattedAddress,places.displayName,places.id,places.location,places.editorialSummary,places.generativeSummary,places.neighborhoodSummary,places.reviews,places.reviewSummary"
  # fieldMask = "*"

  # Make the request
  response = await client.search_text(request=request, metadata=[("x-goog-fieldmask",fieldMask)])
  return response

result = await text_search(f"Burger King Paris")


In [ ]:
result.places

In [9]:
from google.cloud import bigquery


def get_existing_place_ids(project_id, dataset_id, table_id):
    """Retrieves a list of existing place IDs from the BigQuery table."""
    client = bigquery.Client(project=project_id)
    query = f"""
        SELECT DISTINCT place_id
        FROM `{project_id}.{dataset_id}.{table_id}`
    """
    query_job = client.query(query)
    results = query_job.result()  # Wait for the job to complete

    existing_place_ids = [row.place_id for row in results]
    return existing_place_ids



def insert_places_details_row(places):
  # Construct a BigQuery client object.
  client = bigquery.Client(project=PROJECT_ID)

  table_id = f"{PROJECT_ID}.{BIGQUERY_DATASET_ID}.{BIGQUERY_TABLE_PLACE_DETAILS}"

  # Define the table schema
  schema = [
      bigquery.SchemaField("place_id", "STRING", mode="NULLABLE"),
      bigquery.SchemaField("formatted_address", "STRING", mode="NULLABLE"),
      bigquery.SchemaField("display_name", "STRING", mode="NULLABLE"),
      bigquery.SchemaField("location", "GEOGRAPHY", mode="NULLABLE"),
  ]



  # Create the table if it does not exist
  table = bigquery.Table(table_id, schema=schema)
  table = client.create_table(table, exists_ok=True)
  print(f"Table {table.full_table_id}")

  existing_ids = get_existing_place_ids(PROJECT_ID, BIGQUERY_DATASET_ID, BIGQUERY_TABLE_PLACE_DETAILS)
  print(f"Found {len(existing_ids)} existing place IDs in BigQuery.")

  rows_to_insert = []
  for place in places:
    if place.id not in existing_ids:
      location_wkt = f"POINT({place.location.longitude} {place.location.latitude})"

      rows_to_insert.append({
          "place_id": place.id,
          "formatted_address": place.formatted_address,
          "display_name": place.display_name.text,
          "location": location_wkt,

      })

  # Prepare the data for insertion
  if len(rows_to_insert) > 0:
    errors = client.insert_rows_json(table_id, rows_to_insert)
    if errors == []:
        first_add = rows_to_insert[0]["formatted_address"]
        print(f"* New rows ({len(places)}) have been added for {first_add} ")
    else:
        print("Encountered errors while inserting rows: {}".format(errors))
  else:
    print(f"NO row added. Places already exist in database: {len(places)}")



In [8]:

bk_places = [
    "Amiens",
    "Arlay",
    "Bordeaux",
    "Brognon",
    "Buhl-Lorraine",
    "Cernay",
    "Chateaubriant",
    "Châtillon",
    "Cluses",
    "Coquelles",
    "Corbeil-Essonnes",
    "Feurs",
    "Fresnay-l'Évêque",
    "Gasville-Oisème",
    "Gennevilliers",
    "Lançon de Provence",
    "Lille",
    "Limas",
    "Lyon",
    "Mably",
    "Mâcon",
    "Marseille",
    "Merceuil",
    "Metz",
    "Narbonne",
    "Nemours",
    "Nice",
    "Paris",
    "Pierre-Bénite",
    "Reims",
    "Rennes",
    "Saint-Herblain",
    "Saint-Léger",
    "Saran",
    "Taponas",
    "Toulouse",
    "Troyes",
    "Vélizy",
    "Villerbon",
    "Poitier",
    "Limoge",
    "centre",
"Bourg en Bresse",
"Laon",
"Vichy",
"Digne",
"Gap",
"Nice",
"Privas",
"Charleville-Mézièes",
"Foix",
"Troyes",
"Carcassonne",
"Rodez",
"Marseille",
"Caen",
"Aurillac",
"Angoulême",
"La Rochelle",
"Bourges",
"Brive",
"Ajaccio",
"Bastia",
"Dijon",
"Saint Brieuc",
"Guéret",
"Périgueux",
"Besançon",
"Valence",
"Evreux",
"Chartres",
"Brest",
"Nîmes",
"Toulouse",
"Auch",
"Bordeaux",
"Montpellier",
"Rennes",
"Châteauroux",
"Tours",
"Grenoble",
"Lons le Saunier",
"Mont de Marsan",
"Blois",
"Saint Etienne",
"Le Puy",
"Nantes",
"Orléans",
"Cahors",
"Agen",
"Mende",
"Angers",
"Saint-Lô",
"Reims",
"Chaumont",
"Laval",
"Nancy",
"Bar le Duc",
"Vannes",
"Metz",
"Nevers",
"Lille",
"Beauvais",
"Alençon",
"Arras",
"Clermont-Ferrand",
"Pau",
"Tarbes",
"Perpignan",
"Strasbourg",
"Colmar",
"Lyon",
"Lyon",
"Vesoul",
"Macon",
"Le Mans",
"Savoie : 73",
"Annecy",
"Paris",
"Rouen",
"Melun",
"Niort",
"Amiens",
"Albi",
"Montauban",
"Toulon",
"Avignon",
"La Roche sur Yon",
"Poitiers",
"Limoges",
"Epinal",
"Auxerre",
"Belfort",
"Evry",
"Nanterre",
"Bobigny",
"Créteil",
"Pontoise",

              "Paris",
]


distinct_bk_places = sorted(list(set(bk_places)))
print(distinct_bk_places)
print(f"Number of distinct places: {len(distinct_bk_places)}")


for place in bk_places:
  result = await text_search(f"B&B Hotel {place}")
  print(f"Number of places found for {place}: {len(result.places)}")
  insert_places_details_row(result.places)



['Amiens']
Number of distinct places: 1
Number of places found for Amiens: 2
Created table ml-demo-384110:burger_king_reviews_dataset.places_details_v2
Found 1053 existing place IDs in BigQuery.
NO row added. Places already exist in database: 2


Loop through Paris Arrondissements

In [ ]:
for place in range(1, 18):
  search_query = f"Burger King Paris {place}e arrondissement"
  result = await text_search(search_query)
  print(search_query)
  insert_places_details_row(result.places)

Found 145 existing place IDs in BigQuery.
NO row added. Places already exist in database: 1
Found 145 existing place IDs in BigQuery.
NO row added. Places already exist in database: 17
Found 145 existing place IDs in BigQuery.
NO row added. Places already exist in database: 1
Found 145 existing place IDs in BigQuery.
New rows have been added. 19
Found 146 existing place IDs in BigQuery.
NO row added. Places already exist in database: 1
Found 146 existing place IDs in BigQuery.
NO row added. Places already exist in database: 20
Found 146 existing place IDs in BigQuery.
NO row added. Places already exist in database: 20
Found 146 existing place IDs in BigQuery.
NO row added. Places already exist in database: 1
Found 146 existing place IDs in BigQuery.
NO row added. Places already exist in database: 2
Found 146 existing place IDs in BigQuery.
NO row added. Places already exist in database: 2
Found 146 existing place IDs in BigQuery.
NO row added. Places already exist in database: 1
Found 

## Visualize data from Bigquery

In [ ]:
%%bigquery df_places --project ml-demo-384110
SELECT *  FROM `ml-demo-384110.burger_king_reviews_dataset.places_details_v2`


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df_places

,place_id,formatted_address,display_name,location
0,ChIJPcESCsRv5kcRwE1fQz2XnYE,"124 Rue La Boétie, 75008 Paris, France",Burger King,POINT(2.3063735 48.871145)
1,ChIJU-nKqnJu5kcR3nFC67PZNUo,"93 Bd de Strasbourg, 75010 Paris, France",Burger King,POINT(2.3574537 48.8761067)
2,ChIJM69JqB0V5kcRqar8LTrBg-U,"Centre Commercial Paris Nord 2, 117 Av. de la ...",Burger King,POINT(2.499062 48.976546)
3,ChIJsdxKZhhy5kcRo3zVVBTnfwk,"79-81 Quai de la Gare, 75013 Paris, France",Burger King,POINT(2.3743573 48.8365292)
4,ChIJ-YxFBYBv5kcRnQbvMHspZuc,"2 Rue Pierre Lescot, 75001 Paris, France",Burger King,POINT(2.347961 48.8611717)
...,...,...,...,...
149,ChIJS3OStjw1-UcRNHXQh6mLWyU,"6 Rue Francisco Ferrer, 87280 Limoges, France",Burger King,POINT(1.2656772 45.8688209)
150,ChIJi6HKvOHz0YURG_QJmkGEahM,"Av. José López Portillo 220-111-112, San Loren...",Burger King,POINT(-99.0829119 19.6269955)
151,ChIJnUebRsAS2YARRYm6QeHOblM,"Encinos 800, Loma Alta, 21480 Tecate, B.C., Me...",Burger King,POINT(-116.6337287 32.5655338)
152,ChIJu8m813P_0YURApsChvq8dUA,"Av. Insurgentes Sur 729, Nápoles, Benito Juáre...",Burger King,POINT(-99.1730884 19.392522)
